In [1]:
import pandas as pd
import tweepy as tw
import sqlite3
from datetime import datetime
import time

# Tweets collect

In [2]:
#API setting, Tweepy initialize.
BT = 'AAAAAAAAAAAAAAAAAAAAALcmZQEAAAAAg77ypFd9kdFo%2BYrOF3mxDomO4M0%3DeSnYP9qDlLV6RQ7ZlTxNlX0yF8z6LGKxxhg3p4ZUZw2Fj3xmF6'
key = 'd19Es7gYKHmH8BWPDApTyAkMw'
secret = 'rxty3oG2qN6ZafJ5LAU8gKk4WUBaav0KPVuR5e2LqO0CAPaUfX'
TWEEPY = tw.Client(bearer_token=BT,consumer_key=key,consumer_secret=secret,return_type=dict)

In [3]:
def check_table(name,con):
    c = con.cursor()

    #get the count of tables with the name
    c.execute(''' SELECT count(name) FROM sqlite_master WHERE type='table' AND name='{}' '''.format(name))

    #if the count is 1, then table exists
    if c.fetchone()[0]==1 : 
        raise Exception('Date collected')
    else :
        return

In [4]:
def twt_to_sql(Date):
    '''
    This function collects the data from twitter API through tweepy wihtin a certain date,
    and store all tweets with like counts and retweet times into the database.
    The table name is the corresponding date.
    It will first check if the date has already been collected to avoid double collect.
    '''
    #Check if duplicated
    conn = sqlite3.connect('./tweets.db')
    check_table(Date,conn)
    
    #Check if the date is earlier than today
    today = datetime.today()
    if today.strftime("%d/%m/%Y") == pd.to_datetime(Date).strftime("%d/%m/%Y"):
        raise Exception('Collect Tomorrow')
    
    token = None
    flag = 1
    final_df = pd.DataFrame()
    
    #Change date format
    st = str(pd.to_datetime(Date)).replace(' ','T')
    et = str(pd.to_datetime(Date)+pd.DateOffset(1)).replace(' ','T')
    
    #Keep retreiving until no next page
    while (token or flag):
        qry = '(#SP500 OR #SPX500 OR #stocks) lang:en'
        res = TWEEPY.search_recent_tweets(query=qry,
                                          start_time='{}-04:00'.format(st),#Timezone offset applied
                                          end_time='{}-04:00'.format(et),
                                          max_results=100,
                                          tweet_fields='created_at,public_metrics',
                                          next_token=token
                                         )
        df = pd.DataFrame(res['data'])
        df['like'] = df['public_metrics'].apply(lambda x:x['like_count'])
        df['rt'] = df['public_metrics'].apply(lambda x:x['retweet_count'])
        df.drop(columns=['id','public_metrics'],inplace=True)
        final_df = final_df.append(df,ignore_index=True)
        try:
            token = res['meta']['next_token']
        except:
            token = None
        flag = 0
    final_df.dropna(axis=1,inplace=True)
    final_df.to_sql(name=Date,con=conn,if_exists='fail')
    conn.close()

In [5]:
def filt(date):
    conn = sqlite3.connect('./tweets.db')
    df = pd.read_sql('''SELECT * FROM \'{}\''''.format(date),conn)
    conn.close()
    df = df[(df['rt']!=0) & (df['like']!=0) & (df['text'].str.startswith('RT @')==False)].reset_index(drop=True)
    df = df[['text','like','rt']]
    df['date'] = date
    df.to_csv('filtered.csv',mode='a',index=False,header=False)

In [6]:
for i in range(20,24):
    twt_to_sql('2022-04-{}'.format(i))
    filt('2022-04-{}'.format(i))
    time.sleep(450)

In [7]:
df = pd.read_csv('filtered.csv')

In [8]:
df = df[(df['text'].str.contains('https://t.co/')) == False]

In [9]:
conn = sqlite3.connect('./tweets.db')
df = pd.read_sql('''SELECT * FROM \'2022-03-03\'''',conn)
conn.close()

In [10]:
df

,index,text,created_at,like,rt
0,0,"$AVAV - AeroVironment, Inc. 2022 Q3 - Results ...",2022-03-04T04:59:59.000Z,1,0
1,1,Top Gainers #equity #Nifty50 #equities #nifty5...,2022-03-04T04:59:58.000Z,0,0
2,2,#Stocks slide as oil surge kindles inflation f...,2022-03-04T04:59:56.000Z,0,0
3,3,2 Stocks Under $120 You Can Buy and Hold Forev...,2022-03-04T04:59:02.000Z,0,0
4,4,#stocks #StocksMarket \n#UkraineRussianWar Imp...,2022-03-04T04:58:39.000Z,0,0
...,...,...,...,...,...
11887,11887,"RT @leadlagreport: Meanwhile, nearly every spi...",2022-03-03T05:00:18.000Z,0,37
11888,11888,NEWS: $IRNT IronNet Enhances Collective Defens...,2022-03-03T05:00:12.000Z,0,0
11889,11889,#nse #bse #stocks #stockstowatch #stockstobuy ...,2022-03-03T05:00:07.000Z,0,0
11890,11890,Chill out. Shade down. Stream #technology w/ou...,2022-03-03T05:00:05.000Z,2,2


In [11]:
df = df[(df['rt']!=0) & (df['like']!=0) & (df['text'].str.startswith('RT @')==False)].reset_index(drop=True)
df

,index,text,created_at,like,rt
0,12,Downward sloping Angle is meeting with red hor...,2022-03-04T04:57:17.000Z,3,2
1,14,Time to buy Asian Paints. Start accumulation i...,2022-03-04T04:56:59.000Z,1,1
2,30,#ASIANPAINT #Weekly \n\nBroken below the chann...,2022-03-04T04:52:09.000Z,12,2
3,31,#stocks #StocksInFocus \n\n#Hdfcbank from 1390...,2022-03-04T04:52:06.000Z,1,2
4,32,➡️DART Bullish on #APTUS\n\n⚡Buy | TP: 475 | U...,2022-03-04T04:51:56.000Z,10,2
...,...,...,...,...,...
916,11872,Chill out. Shade down. Stream #technology w/ou...,2022-03-03T05:03:21.000Z,2,1
917,11876,Chill out. Shade down. Stream #technology w/ou...,2022-03-03T05:01:53.000Z,2,1
918,11878,Thought for the Day!\nFollow us @viewprosperit...,2022-03-03T05:01:30.000Z,1,1
919,11885,Chill out. Shade down. Stream #technology w/ou...,2022-03-03T05:00:32.000Z,2,2


In [12]:
df = df[(df['text'].str.contains('https://t.co/')) == False]
df

,index,text,created_at,like,rt
1,14,Time to buy Asian Paints. Start accumulation i...,2022-03-04T04:56:59.000Z,1,1
3,31,#stocks #StocksInFocus \n\n#Hdfcbank from 1390...,2022-03-04T04:52:06.000Z,1,2
6,59,#NSE BUY #PIDILITIND 2267.45 - MAC2\n\r\r🟢 #Au...,2022-03-04T04:47:09.000Z,1,3
9,74,#NSE BUY #BATAINDIA 1732.35 - MIC2\n\r\r🟢 #Aut...,2022-03-04T04:45:13.000Z,1,2
10,103,#NSE BUY #HINDUNILVR 2028.7 - MAC3\n\r\r🟢 #Aut...,2022-03-04T04:39:48.000Z,1,1
...,...,...,...,...,...
897,11742,"Want to trade in US stocks like #Amazon, Tesla...",2022-03-03T05:23:37.000Z,4,3
898,11744,For Penny Stocks Lovers\n#FcsSoft\nCmp 3.6\n\n...,2022-03-03T05:23:30.000Z,10,3
901,11769,Supriya Life Sciences\nCMP :- 425\nTarget :- 4...,2022-03-03T05:19:20.000Z,1,1
903,11803,"Sugar #stocks sweeten market mood; Balrampur, ...",2022-03-03T05:13:42.000Z,1,1
